In [39]:
# autoreload
%load_ext autoreload
%autoreload 2

import os
if not "milligrad" in os.listdir():
    os.chdir('..')

from milligrad import Tensor, Module, Adam, topological_sort
from extras.mnist import load_mnist_data

import numpy as np
from matplotlib import pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
(x_train, y_train), (x_test, y_test) = load_mnist_data()

x_train, x_test = x_train.reshape(-1, 28*28), x_test.reshape(-1, 28*28)
y_train, y_test = np.eye(10)[y_train], np.eye(10)[y_test]

x_train, x_test = x_train.astype("float32"), x_test.astype("float32")


In [49]:
from milligrad.tensor import Tensor


class MLP(Module):
    def __init__(self):
        super().__init__()
        self.w1 = Tensor.xavier(784, 128)
        self.b1 = Tensor.randn(128)
        self.w2 = Tensor.xavier(128, 10)
        self.b2 = Tensor.randn(10)
        
    def __call__(self, x:Tensor)->Tensor:
        x = (x @ self.w1 + self.b1).relu()
        return x @ self.w2 + self.b2
    
    def parameters(self)->list[Tensor]:
        return [self.w1, self.b1, self.w2, self.b2]

In [51]:
mlp = MLP()
opt = Adam(mlp.parameters(), lr=1e-3)

for i in range(100):
    y_hat = mlp(Tensor(x_train[:10]))
    loss = y_hat.nll_loss(Tensor(y_train[:10]))
    
    opt.zero_grad()
    loss.backward()
    opt.step()
    
    print(loss.data.mean())

IndexError: arrays used as indices must be of integer (or boolean) type